In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
os.chdir("drive/My Drive/PROJECT/Sentiment/")

In [ ]:
# Run only time
!git clone https://github.com/datquocnguyen/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 42758 (delta 3), reused 11 (delta 3), pack-reused 42744
Receiving objects: 100% (42758/42758), 30.66 MiB | 7.30 MiB/s, done.
Resolving deltas: 100% (29589/29589), done.
Checking out files: 100% (1216/1216), done.


In [ ]:
cd transformers

/content/drive/My Drive/PROJECT/Sentiment/transformers


In [ ]:
!pip install .

Processing /content/drive/My Drive/PROJECT/Sentiment/transformers
     |████████████████████████████████| 3.0MB 6.4MB/s 
     |████████████████████████████████| 1.1MB 53.2MB/s 
     |████████████████████████████████| 890kB 52.8MB/s 
  Created wheel for transformers: filename=transformers-3.1.0-cp36-none-any.whl size=996650 sha256=4346476937c2b1143e451a00bda82a011893f516eb44940923bb73be9e1174b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-8tpfe6d6/wheels/d5/74/76/97852b1741ddb1c63040c4614e89114b041cd92ada9f6fa1d8
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=4ecd01da9f3bddb86de5b515caee0c6d520b18592ee1dc79a74cd424ae55483b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built transformers sacremoses


In [ ]:
%cd ..

/content/drive/My Drive/PROJECT/Sentiment


In [ ]:
# Run only time
!git clone https://github.com/vncorenlp/VnCoreNLP.git

Cloning into 'VnCoreNLP'...
remote: Enumerating objects: 212, done.
remote: Total 212 (delta 0), reused 0 (delta 0), pack-reused 212
Receiving objects: 100% (212/212), 214.21 MiB | 25.77 MiB/s, done.
Resolving deltas: 100% (76/76), done.
Checking out files: 100% (33/33), done.


In [ ]:
!pip install vncorenlp

     |████████████████████████████████| 2.7MB 5.2MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-cp36-none-any.whl size=2645934 sha256=faf175a677860a60674a3a0ae192be71fba371e5795edd75fed1b183a0dc7443
  Stored in directory: /root/.cache/pip/wheels/09/54/8b/043667de6091d06a381d7745f44174504a9a4a56ecc9380c54
Successfully built vncorenlp


In [ ]:
import json
import numpy as np
import pandas as pd
import re

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel
from vncorenlp import VnCoreNLP
from sklearn.model_selection import train_test_split

from tqdm import tqdm

In [ ]:
batch_size = 32
max_length = 256
if torch.cuda.is_available():       
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
# Define model
class SentimentModel(nn.Module):
    def __init__(self):
        super(SentimentModel, self).__init__()
        self.phoBert = AutoModel.from_pretrained('vinai/phobert-base')
        self.linear_1 = nn.Linear(768, 100)
        self.linear_2 = nn.Linear(100, 2)
        
    def forward(self, input_ids, attention_mask):
        output = self.phoBert(input_ids, attention_mask)[1]
        x = self.linear_1(output)
        x = self.linear_2(x)
        return x

In [ ]:
# tokenize function
annotator = VnCoreNLP('./VnCoreNLP/VnCoreNLP-1.1.1.jar', annotators='wseg', max_heap_size='-Xmx2g')
bpe = AutoTokenizer.from_pretrained('vinai/phobert-base')
def tokenize_and_preprocess(text):
    only_words = re.findall('\w+', text)
    text = " ".join(only_words)
    if len(text) > 0:
        text = annotator.tokenize(text)[0]
        return " ".join(text).lower()
    else:
        return 'NULL'

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
def load_data(test_size=500):
    df = pd.read_excel('train.xlsx', encoding='utf-8')
    datas = list(df['text'].apply(tokenize_and_preprocess))
    labels = list(df['label'])
    train_data, test_data, train_label, test_label = train_test_split(datas, labels, test_size=test_size, random_state=42)
    return train_data, test_data, train_label, test_label

In [ ]:
train_data, test_data, train_label, test_label = load_data()
X_train = bpe.batch_encode_plus(train_data, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
y_train = torch.tensor(train_label, dtype=torch.long)
X_test = bpe.batch_encode_plus(test_data, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
y_test = torch.tensor(test_label, dtype=torch.long)

In [ ]:
# Test tokenization
print(bpe.convert_ids_to_tokens(X_train['input_ids'][4]))

['<s>', 'Mình', 'hơi', 'thất', 'vọng', '1', 'chút', 'vì', 'mình', 'đã', 'kỳ', 'vọng', 'cuốn', 'sách', 'khá', 'nhiều', 'hi', 'vọng', 'nó', 'sẽ', 'nói', 'về', 'việc', 'học', 'tập', 'của', 'cách', 'sinh', 'viên', 'trường', 'Harvard', 'ra', 'sao', 'những', 'n@@', 'ỗ', 'lực', 'của', 'họ', 'như', 'thế', 'nào', '4h', 'sáng', 'tại', 'sao', 'họ', 'lại', 'phải', 'thức', 'dậy', 'vào', 'thời', 'khắc', 'đấy', 'sau', 'đó', 'là', 'cả', 'một', 'câu', 'chuyện', 'ra', 'sao', 'Cái', 'mình', 'thực', 'sự', 'cần', 'ở', 'đây', 'là', 'câu', 'chuyện', 'ẩn', 'dấu', 'trong', 'đó', 'để', 'tự', 'bản', 'thân', 'mỗi', 'người', 'cảm', 'nhận', 'và', 'đi', 'sâu', 'vào', 'lòng', 'người', 'hơn', 'Còn', 'cuốn', 'sách', 'này', 'chỉ', 'đơn', 'thuần', 'là', 'cuốn', 'sách', 'dạy', 'kĩ', 'năng', 'mà', 'hầu', 'như', 'sách', 'nào', 'cũng', 'đã', 'có', 'B@@', 'U@@', 'ồn', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>',

In [ ]:
train_tensor = TensorDataset(X_train['input_ids'], X_train['attention_mask'], y_train)
train_loader = DataLoader(train_tensor, batch_size=batch_size, shuffle=True)

test_tensor = TensorDataset(X_test['input_ids'], X_test['attention_mask'], y_test)
test_loader = DataLoader(test_tensor, batch_size=batch_size, shuffle=False)

In [ ]:
# Init model and optimizer & schedule
def initialize_model(lr=3e-5):
    model = SentimentModel()
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=lr)
    n_steps = len(train_loader) * 5
    scheduler = get_linear_schedule_with_warmup(optimizer=optimizer,
                                                num_warmup_steps=50,
                                                num_training_steps=n_steps)
    #loss_fn = nn.BCEWithLogitsLoss()
    loss_fn = nn.CrossEntropyLoss()
    return model, optimizer, scheduler, loss_fn

In [ ]:
def step(model, optimizer, scheduler, loss_fn, batch):
    model.train()
    input_ids, attention_mask, label = tuple(t.to(device) for t in batch)
    optimizer.zero_grad()

    y_pred = model.forward(input_ids, attention_mask)
    loss = loss_fn(y_pred, label)
    loss.backward()

    optimizer.step()
    scheduler.step()

    return loss.item()

In [ ]:
def validate(model, loss_fn, test_loader=test_loader):
    print("Evaluating....")
    model.eval()
    with torch.no_grad():
        total_loss = 0.0
        accuracy = 0
        for i, batch in enumerate(test_loader):
            input_ids, attention_mask, y_true = tuple(t.to(device) for t in batch)
            y_pred = model.forward(input_ids, attention_mask)
            loss = loss_fn(y_pred, y_true)

            total_loss += loss.item()
            accuracy += (y_pred.argmax(1) == y_true).sum().item()

        val_loss = total_loss/len(test_loader)
        accuracy = accuracy/len(test_tensor) 
    return val_loss, accuracy

In [ ]:
def train(model, optimizer, scheduler, loss_fn, epochs=5):
    for epoch in range(epochs):
        total_loss = 0.0
        for i, batch in enumerate(train_loader):
            loss = step(model, optimizer, scheduler, loss_fn, batch)
            total_loss += loss 
            if (i + 1) % 50 == 0:
                print("Epoch: {} - iter: {}/{} - train_loss: {}".format(epoch, i + 1, len(train_loader), total_loss/(i + 1)))
        val_loss, accuracy = validate(model, loss_fn)
        print("Epoch: {}/{} - val_loss: {} - accuracy: {}".format(epoch, epochs, val_loss, accuracy))

In [ ]:
model, optimizer, scheduler, loss_fn = initialize_model()
print("Starting training...")
train(model, optimizer, scheduler, loss_fn, epochs=3)
print("Saving model...")
torch.save(model.state_dict(), 'model_no_tokenize.pt')


Starting training...
Epoch: 0 - iter: 50/488 - train_loss: 0.6187833154201507
Epoch: 0 - iter: 100/488 - train_loss: 0.5044060000777244
Epoch: 0 - iter: 150/488 - train_loss: 0.48482668936252593
Epoch: 0 - iter: 200/488 - train_loss: 0.44072315603494644
Epoch: 0 - iter: 250/488 - train_loss: 0.41753866875171664
Epoch: 0 - iter: 300/488 - train_loss: 0.3941977071513732
Epoch: 0 - iter: 350/488 - train_loss: 0.3782069624534675
Epoch: 0 - iter: 400/488 - train_loss: 0.365020822416991
Epoch: 0 - iter: 450/488 - train_loss: 0.35368720668885445
Evaluating....
Epoch: 0/3 - val_loss: 0.46260460186749697 - accuracy: 0.792
Epoch: 1 - iter: 50/488 - train_loss: 0.3560541942715645
Epoch: 1 - iter: 100/488 - train_loss: 0.31423541508615016
Epoch: 1 - iter: 150/488 - train_loss: 0.29513026371598244
Epoch: 1 - iter: 200/488 - train_loss: 0.28746525023132563
Epoch: 1 - iter: 250/488 - train_loss: 0.28268255934119224
Epoch: 1 - iter: 300/488 - train_loss: 0.27868833375473817
Epoch: 1 - iter: 350/488 -

In [ ]:
df = pd.read_excel('test.xlsx', encoding='utf-8')
#Read file
indices = list(df['id'])
texts = list(df['text'].apply(tokenize_and_preprocess))
# Encode
X_test_input = bpe.batch_encode_plus(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')

In [ ]:
labels = []
model.eval()
with torch.no_grad():
    for i in range(0, len(X_test_input['input_ids'])):
        input_ids_i = X_test_input['input_ids'][i]
        attention_mask_i = X_test_input['attention_mask'][i]
        y_p = model.forward(input_ids_i.unsqueeze(0).to(device), attention_mask_i.unsqueeze(0).to(device))
        pred = y_p.argmax(1).squeeze()
        labels.append(int(pred))
df_test = {'id': indices, 'label': labels}
df_test = pd.DataFrame(df_test, columns=['id', 'label'])
df_test.to_csv('submission.csv', index=False, encoding='utf-8')